# 03. PyTorch Computer Vision

**What is a computer vision problem?**: Anything that you can see
 - Binary classification problem (distinguishing images from two classes)
 - Multi-class classification problem (distinguishing images form among multiple classes)
 - Object detection (find a target object within an image)
 - Image segmentation (split an image into multiple context segments that make up an image)
 - Environmental modelling/perception (e.g. for self-driven automobiles)

**Items covered in this section**

![](./screenshots/1_topics_covered_computer_vision_module.png)

**Inputs and outputs of a computer vision algorithm**

![](./screenshots/2_Compter_vision_inputs_and_outputs.png)

**Input and output shapes** (for a classification example)

 - **Input:** `[batch_size, height, width, color_channels]`
 - **Output:** `[number_of_classes]`

This can be done according to various conventions:

 - **NHWC:** `[batch_size, height, width, color_channels]`
 - **NCHW** `[batch_size, color_channels, height, width]`

One needs to keep in mind that data from different sources can be shaped differently, which needs to be taken into account in the data preparation code.

![](./screenshots/3_Input_and_output_shapes.png)

**PyTorch workflow**

![](./screenshots/4_PyTorch_workflow.png)